In [44]:
# from keras.layers.embeddings import Embedding
# from keras.preprocessing.text import Tokenizer
from nltk import word_tokenize
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import re



from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Concatenate, Attention
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from nltk import word_tokenize


from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Convolution1D
# from keras.layers import LSTM

from nltk.stem import WordNetLemmatizer 



from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.base import BaseEstimator

from tqdm.notebook import tqdm


from sklearn.preprocessing import StandardScaler
import string
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from statistics import mean


### Build embedding and df

In [2]:
embeddings_dictionary = dict()
glove_file = open('glove.txt', encoding="utf8")

for line in tqdm(glove_file):
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [6]:
df = pd.read_csv("train_data.csv")
df.sample(2)

,id,keyword,location,text,target
1284,1852,burned,DC,burned 202 calories doing 30 minutes of Walkin...,0
2107,3026,death,"Sylacauga, Alabama",@Allahsfinest12 ...death to muslims,1


### define functions

In [3]:
tknzr = TfidfVectorizer(encoding = 'latin1').build_analyzer()
stop_words = stopwords.words('english')
# Utillity functions

def word_count(str_in):
    return len(str_in.split())

def unique_word_count(str_in):
    return len(set(str_in.split()))

def stop_word_count(str_in):
    return len([w for w in tknzr(str_in) if w in stop_words])

def mean_word_length(str_in):
    # return round(sum([len(w) for w in str_in.split()])/len(str_in.split()),0)
    return mean([len(w) for w in str_in.split()])

def get_tweets_features(df_in, col='text'):
    """ returns a df with tweets stats"""
    
    url_reg = r'(?:^|\s)https?://\S*'
    punc_reg = r'[{}]'.format(string.punctuation)
    alpha_reg = r'[a-zA-Z]'
    numeric_reg = r'[0-9]'
    
    df_out = pd.DataFrame()
    
    df_out['n_words'] = df_in[col].apply(word_count)
    df_out['n_setwords'] = df_in[col].apply(unique_word_count)
    df_out['n_stopwords'] = df_in[col].apply(stop_word_count)
    df_out['mean_word_len'] = df_in[col].apply(mean_word_length)

    df_out['n_#'] = df_in[col].str.count('#')
    df_out['n_@'] = df_in[col].str.count('@')
    df_out['n_chars'] = df_in[col].str.len()
    df_out['n_url'] = df_in[col].str.count(url_reg)
    df_out['n_punc'] = df_in[col].str.count(punc_reg)
    df_out['n_alpha'] = df_in[col].str.count(alpha_reg)
    df_out['n_numeric'] = df_in[col].str.count(numeric_reg)
    
    return df_out


def category_analysis(df_in, col, target, min_occur=0):
    """
    input:  df_in - dataframe which contains the columns: col and target
                    the categories in col are assumed to be processed. 
            min_occur - number of times the class appears (by default = 10)
    
    """
    df_classes = pd.DataFrame()
    df_classes['count'] = df_in.groupby(col)[target].count().sort_values(
        ascending=False)

    for t in df_in[target].unique():
        df_classes[t] = df_in[df_in[target]==t].groupby(col)[target].count()
        
    df_classes = df_classes[df_classes['count'] > min_occur]
    df_classes.fillna(value=0, inplace=True)
    return df_classes

In [4]:
def clean_text_norm(df, col='text'):
    s_clean = df[col].copy()
    s_clean = s_clean.str.replace('http\S+', '__URL__', flags= re.IGNORECASE, regex=True)   
    s_clean = s_clean.str.replace('\@\S+', '__M__', regex=True)
    s_clean = s_clean.str.replace('#\S+','__H__', regex=True)
    s_clean = s_clean.str.replace('[0-9]{1,2}(/|-)[0-9]{1,2}(/|-)[0-9]{2,4}','__DATE__', regex=True)
    s_clean = s_clean.str.replace('20[0-9]{2}','__YEAR__', regex=True)
    s_clean = s_clean.str.replace('[0-9]{1,2}(:)[0-9]{1,2}((:)[0-9]{1,2})?( [UTC|PM|AM])?','__TIME__', flags= re.IGNORECASE, regex=True)
    s_clean = s_clean.str.replace('\S+Û\S+','__U__', flags= re.IGNORECASE, regex=True)
    s_clean = s_clean.str.replace('\d+(\-|\.)?\d?(%)?','__d__', regex=True)
    s_clean = s_clean.str.replace('&amp(;)?','__AMP__', regex=True)
    return s_clean

In [11]:
class Lemmatizer(BaseEstimator):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.analyzer = TfidfVectorizer().build_analyzer()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if type(X) != pd.Series:
            X = pd.Series(X)
        return X.apply(lambda s: self._get_lemma_words(s)) 
    
    def _get_lemma_words(self, sentance):
        words = self.analyzer(sentance)
        return ' '.join([self.lemmatizer.lemmatize(word) for word in words])

### Build the features

In [24]:
lama = Lemmatizer()
df["text"] = lama.transform(df["text"])

In [25]:
x = df["text"]
y = df["target"]

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [27]:
y_train = y_train.values
y_test = y_test.values
x_train = x_train.to_frame()
x_test = x_test.to_frame()

In [29]:
meta_train = get_tweets_features(x_train)
meta_test = get_tweets_features(x_test)

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train["text"])
x_train = tokenizer.texts_to_sequences(x_train["text"])
x_test = tokenizer.texts_to_sequences(x_test["text"])

In [33]:
vocab_size = len(tokenizer.word_index) + 1
maxlen = 200
padding='post'

In [34]:
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [35]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [36]:
batch_size =32
embedding_dim = 100
filters= 2
kernel_size = 3
hidden_dim = 64
dense_dim1 = 128
dense_dim2 = 256
meta_size = len(meta_train.columns)
output_dim = 1
maxlen = maxlen

In [ ]:
# Variable-length int sequences.
query_input = tf.keras.Input(shape=(None,), dtype='int32')
value_input = tf.keras.Input(shape=(None,), dtype='int32')

# Embedding lookup.
token_embedding = tf.keras.layers.Embedding(max_tokens, dimension)
# Query embeddings of shape [batch_size, Tq, dimension].
query_embeddings = token_embedding(query_input)
# Value embeddings of shape [batch_size, Tv, dimension].
value_embeddings = token_embedding(query_input)

# CNN layer.
cnn_layer = tf.keras.layers.Conv1D(
    filters=100,
    kernel_size=4,
    # Use 'same' padding so outputs have the same shape as inputs.
    padding='same')
# Query encoding of shape [batch_size, Tq, filters].
query_seq_encoding = cnn_layer(query_embeddings)
# Value encoding of shape [batch_size, Tv, filters].
value_seq_encoding = cnn_layer(value_embeddings)

# Query-value attention of shape [batch_size, Tq, filters].
query_value_attention_seq = tf.keras.layers.Attention()(
    [query_seq_encoding, value_seq_encoding])

# Reduce over the sequence axis to produce encodings of shape
# [batch_size, filters].
query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    query_seq_encoding)
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

# Concatenate query and document encodings to produce a DNN input layer.
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

In [47]:
input_holder = Input(shape=(maxlen,))

layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
              weights=[embedding_matrix], input_length=maxlen, mask_zero=True, 
              trainable=False)(input_holder) #trainable=False - do not touch this label

# layer = Convolution1D(filters=filters,kernel_size=kernel_size, activation='relu')(layer) #next line
# layer = MaxPooling1D()(layer) max val_accuracy: 0.564
layer = LSTM(units=hidden_dim)(layer)

# layer = Attention(name='attention_layer')(layer)


layer = Dense(units = dense_dim1,activation='relu')(layer)

input_holder_meta = Input(shape=(meta_size,))

layer = Concatenate()([layer,input_holder_meta])

layer = Dense(units = dense_dim2,activation='relu')(layer)
layer = Dense(units = output_dim,activation='sigmoid')(layer)

In [48]:
model = Model(inputs=[input_holder,input_holder_meta], outputs=layer)

In [49]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 200, 100)     1547800     input_5[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 64)           42240       embedding_3[0][0]                
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          8320        lstm_3[0][0]                     
____________________________________________________________________________________________

In [50]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"],)

In [51]:
history = model.fit(x=[x_train,meta_train],y=y_train,validation_split=0.3,epochs=50,batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', min_delta=0.005,patience=5,verbose=1)])

Train on 3570 samples, validate on 1530 samples
Epoch 1/50
3570/3570 [==============================] - 61s 17ms/sample - loss: 0.7108 - accuracy: 0.6754 - val_loss: 0.4771 - val_accuracy: 0.7830
Epoch 2/50
3570/3570 [==============================] - 52s 14ms/sample - loss: 0.4444 - accuracy: 0.8050 - val_loss: 0.4860 - val_accuracy: 0.7941
Epoch 3/50
3570/3570 [==============================] - 50s 14ms/sample - loss: 0.4239 - accuracy: 0.8118 - val_loss: 0.4687 - val_accuracy: 0.7954
Epoch 4/50
3570/3570 [==============================] - 51s 14ms/sample - loss: 0.4225 - accuracy: 0.8132 - val_loss: 0.4962 - val_accuracy: 0.7863
Epoch 5/50
3570/3570 [==============================] - 49s 14ms/sample - loss: 0.3864 - accuracy: 0.8364 - val_loss: 0.5433 - val_accuracy: 0.7830
Epoch 6/50
3570/3570 [==============================] - 49s 14ms/sample - loss: 0.3433 - accuracy: 0.8515 - val_loss: 0.6104 - val_accuracy: 0.7458
Epoch 7/50
3570/3570 [==============================] - 51s 14ms

In [96]:
model.evaluate(x=[x_test,meta_test],y=y_test)

2513/2513 [==============================] - 8s 3ms/sample - loss: 0.4976 - accuracy: 0.7875


[0.49756767405393443, 0.787505]

In [139]:
predictions = []
index = 1
for index in range (len(x_test)):
    predict = model.predict(x=[np.expand_dims(x_test[0], axis=0),meta_test.iloc[index].to_frame().T],)

    predictions.append(predict[0][0])


In [142]:
[x for x in predictions if x>=0.5]

[0.882954, 0.8310261, 0.8408423, 0.5073233, 0.5737174, 0.8572335, 0.8262726]